<a href="https://colab.research.google.com/github/rebornrulz/Rulz-AI/blob/master/colabs/tensorflow/DL_Seq2Seq_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.2.0

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow==2.2.0 (from versions: 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.15.0rc0, 2.15.0rc1)
ERROR: No matching distribution found for tensorflow==2.2.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip uninstall keras-nightly

In [3]:
pip install Keras==2.3.1

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.8/377.8 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 19.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
%tensorflow_version 1.14.0 import tensorflow print(tensorflow.version)

UsageError: Line magic function `%tensorflow_version` not found.


In [5]:
# Code referred from https://github.com/DwikiWitman/Chatbot-Bahasa-Indonesia
#attention_decoder.py
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

def _time_distributed_dense(x, w, b=None, dropout=None,
                            input_dim=None, output_dim=None,
                            timesteps=None, training=None):
    if not input_dim:
        input_dim = K.shape(x)[2]
    if not timesteps:
        timesteps = K.shape(x)[1]
    if not output_dim:
        output_dim = K.shape(w)[1]

    if dropout is not None and 0. < dropout < 1.:
        # apply the same dropout pattern at every timestep
        ones = K.ones_like(K.reshape(x[:, 0, :], (-1, input_dim)))
        dropout_matrix = K.dropout(ones, dropout)
        expanded_dropout_matrix = K.repeat(dropout_matrix, timesteps)
        x = K.in_train_phase(x * expanded_dropout_matrix, x, training=training)

    # collapse time dimension and batch dimension together
    x = K.reshape(x, (-1, input_dim))
    x = K.dot(x, w)
    if b is not None:
        x = K.bias_add(x, b)
    # reshape to 3D tensor
    if K.backend() == 'tensorflow':
        x = K.reshape(x, K.stack([-1, timesteps, output_dim]))
        x.set_shape([None, None, output_dim])
    else:
        x = K.reshape(x, (-1, timesteps, output_dim))
    return x

class AttentionDecoder(Recurrent):

    def __init__(self, units, output_dim,
                 activation='softmax',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):

        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = _time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        print('Inputs Shape:', inputs.get_shape())

        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import csv #preprocessing.py
import os
import re

# procedure clean noise dataset
def preprocessing_text(text):
    text = text.lower()

    text = re.sub(r'[^a-zA-Z\s.,?!]', u'', text, flags=re.UNICODE)


    aku = ['ku', 'aku']
    nuan =['dek']
    iya = ['ya']
    kami = ['kita']
    enda = ['enda', 'nda', 'nadai', 'ukai']
    iya = ['ya', 'sanu']
    hai = ['hi', 'hey', 'halo', 'hay']

    lines = []
    for word in text.split():
        if word in aku:
            lines.append("ku")
        elif word in nuan:
            lines.append("dek")
        elif word in iya:
            lines.append("ya")
        elif word in kami:
            lines.append("kita")
        elif word in enda:
            lines.append("nadai")
        elif word in hai:
            lines.append("hi")
        else:
            lines.append(word)

    text = ' '.join(lines)

    text = ' '.join(text.split())

    maxlen = 15
    if len(text.split()) > maxlen:
        text = (' ').join(text.split()[:maxlen])

    return text

# read Corpus
lines_filepath = os.path.join("Corpus","Iban.tsv")
with open(lines_filepath, "r", encoding="utf-8") as lines:
    array = []
    for line in lines:
        line = preprocessing_text(line.rstrip('\n'))
        array.append(line)
lines.close()
# write context-target
lines_filepath = os.path.join("Corpus","Iban.tsv")
with open(lines_filepath, "w", encoding="utf-8",newline='') as lines:
    writer = csv.writer(lines, delimiter='\t')
    i = 0
    while i < len(array):
        try:
            writer.writerow([array[i], array[i+1]])
        except:
            pass
        i+=1
lines.close()


In [ ]:
pip install tensorflow==2.2.0

  Using cached https://files.pythonhosted.org/packages/4c/1a/0d79814736cfecc825ab8094b39648cc9c46af7af1bae839928acb73b4dd/tensorflow-2.2.0-cp37-cp37m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/1d/74/0a6fcb206dcc72a6da9a62dd81784bfdbff5fedb099982861dc2219014fb/tensorboard-2.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a4/f5/926ae53d6a226ec0fda5208e0e581cffed895ccc89e36ba76a8e60895b78/tensorflow_estimator-2.2.0-py2.py3-none-any.whl
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.2
    Uninstalling tensorflow-1.15.2:
      Successfully uninstalled tensorflow-1.15.2


In [ ]:
import csv
import os
import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
#from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dense , Dropout, Activation, Permute, Multiply,SimpleRNN
from keras.layers.normalization import BatchNormalization
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.optimizers import Adam

from keras.callbacks import TensorBoard
from attention_decoder import AttentionDecoder
from keras import backend as K

file = "Iban.tsv"
folder = "Corpus"
# load a clean dataset
def load_clean_sample_data(folder, file):
    #return load(open(filename, 'rb'))
    lines_filepath = os.path.join(folder,file)
    with open(lines_filepath, "r", encoding="utf8") as read:
        reader = csv.reader(read,delimiter="\t")
        dataset = []
        for row in reader:
            dataset.append(row)
    read.close()
    return dataset

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # intereply encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    result_precision = precision(y_true, y_pred)
    result_recall = recall(y_true, y_pred)
    return 2*((result_precision*result_recall)/(result_precision+result_recall+K.epsilon()))


def define_model(vocab, timesteps, n_units, encoder, decoder, attention):
    model = Sequential()
    model.add(Embedding(vocab, n_units, input_length=timesteps, mask_zero=True))
    #model.add(Embedding(vocab, n_units, weights=[embedding_vectors], input_length=timesteps, trainable=False))
    if(encoder == "LSTM"):
        model.add(LSTM(n_units, return_sequences=False, dropout=0.5, recurrent_dropout=0.5))

    model.add(RepeatVector(timesteps))
    if(decoder == "LSTM"):
        model.add(LSTM(n_units, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    model.add(BatchNormalization())
    if(attention == "ATTNDECODER"):
        model.add(AttentionDecoder(n_units, vocab))

    return model

# define NMT model
def define_model(vocab, timesteps, n_units, encoder, decoder, attention):
    model = Sequential()
    model.add(Embedding(vocab, n_units, input_length=timesteps, mask_zero=True))
    #model.add(Embedding(vocab, n_units, weights=[embedding_vectors], input_length=timesteps, trainable=False))
    model.add(SimpleRNN(n_units, return_sequences=False))
    model.add(RepeatVector(timesteps))
    model.add(SimpleRNN(n_units, return_sequences=True))
    #model.add(BatchNormalization())
    if(attention == "ATTNDECODER"):
        model.add(AttentionDecoder(n_units, vocab))
    else:
        model.add(TimeDistributed(Dense(vocab, activation='tanh',
                                        #kernel_regularizer=regularizers.l2(0.01),
                                        #activity_regularizer=regularizers.l2(0.01)
                                        )))
    return model

#load datasets
dataset = load_clean_sample_data("Corpus","Iban.tsv")
dataset = np.reshape(dataset, (-1,2))
dataset1 = dataset.reshape(-1,1)
train, test = dataset[ : int(len(dataset)*80/100) ],  dataset[ int(len(dataset)*80/100): ]
del dataset

# prepare tokenizer
all_tokenizer = create_tokenizer(dataset1[:, 0])
print(all_tokenizer)
all_vocab_size = len(all_tokenizer.word_index) + 1
all_length = max_length(dataset1[:, 0])
print('ALL Vocabulary Size: %d' % (all_vocab_size))
print('ALL Max question length: %d' % (all_length))
del dataset1

# prepare training data
trainX = encode_sequences(all_tokenizer, all_length, train[:, 0])
trainY = encode_sequences(all_tokenizer, all_length, train[:, 1])
trainY = encode_output(trainY, all_vocab_size)
del train

# prepare validation data
testX = encode_sequences(all_tokenizer, all_length, test[:, 0])
testY = encode_sequences(all_tokenizer, all_length, test[:, 1])
testY = encode_output(testY, all_vocab_size)
del test

# define model (yg bagian ini yg diedit sesuai kemauan dgn menyesuaikan pemanggilan function2 diatas)
#raw_embedding = load_embedding('Word2vec/idwiki_word2vec.txt')
#embedding_vectors = get_weight_matrix(raw_embedding, all_tokenizer.word_index)
#model = define_model_embed(all_vocab_size, all_length, 100, encoder, decoder, embedding_vectors)
encoder = "LSTM"
decoder = "LSTM"
attention = "ATTNDECODER"
model = define_model(all_vocab_size, all_length, 256, encoder, decoder, attention)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy', precision, recall, f1])
#model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics = ['accuracy', precision, recall, f1])

# summarize defined model
print(model.summary())
#tensorboard --logdir ./Graph
#tensorboard --logdir=LSTM-LSTM:./Model/15rb/lstm-lstm,LSTM-GRU:./Model/15rb/lstm-gru,GRU-GRU:./Model/15rb/gru-gru,GRU-LSTM:./Model/15rb/gru-lstm
# tensor_board = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
#train and save model
import timeit
start_time = timeit.default_timer()
history = model.fit(trainX, trainY, epochs=100, batch_size=32, validation_data=(testX, testY), verbose=1)
print(timeit.default_timer() - start_time)

import timeit
start_time = timeit.default_timer()
score = model.evaluate(testX, testY, batch_size=64)
print(timeit.default_timer() - start_time)
print(score)

import datetime
str(datetime.timedelta(seconds=3437))

filename = 'Model/MODEL_'+encoder+'_'+decoder+'_'+attention+'.h5'
model.save(filename)

ALL Vocabulary Size: 740
ALL Max question length: 15
Inputs Shape: (None, 15, 256)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 256)           189440    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               131328    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 15, 256)           0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 15, 256)           131328    
_________________________________________________________________
AttentionDecoder (AttentionD (None, 15, 740)           2086644   
Total params: 2,538,740
Trainable params: 2,538,740
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 599 samples, validate on 150 samples
Epoch 1/100
599/599 [==============================] - 9s 16ms/step - loss: 5.0127 - accuracy: 0.1372 - precision: 0.1329 - recall: 0.0031 - f1: 0.0060 - val_loss: 5.3062 - val_accuracy: 0.2164 - val_precision: 0.3246 - val_recall: 0.0112 - val_f1: 0.0213
Epoch 2/100
599/599 [==============================] - 7s 12ms/step - loss: 4.2100 - accuracy: 0.1944 - precision: 0.5270 - recall: 0.0167 - f1: 0.0320 - val_loss: 4.9473 - val_accuracy: 0.3209 - val_precision: 0.3023 - val_recall: 0.0471 - val_f1: 0.0813
Epoch 3/100
599/599 [==============================] - 7s 13ms/step - loss: 4.1062 - accuracy: 0.1903 - precision: 0.5927 - recall: 0.0207 - f1: 0.0399 - val_loss: 5.1341 - val_accuracy: 0.3187 - val_precision: 0.2915 - val_recall: 0.0331 - val_f1: 0.0591
Epoch 4/100
599/599 [==============================] - 8s 13ms/step - loss: 4.0401 - accuracy: 0.2007 - precision: 0.4973 - recall: 0.0214 - f1: 0.0407 - val_loss: 5.4305 - val_accuracy:

In [ ]:
import numpy as np #chatting.py
import csv
import os
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from attention_decoder import AttentionDecoder
from keras import backend as K

file = "Iban.tsv"
folder = "Corpus"

# load a clean dataset
def load_clean_sample_data():
    #return load(open(filename, 'rb'))
    filepath = os.path.join(folder,file)
    with open(filepath, "r", encoding="utf8") as read:
        reader = csv.reader(read,delimiter="\t")
        dataset = []
        for row in reader:
            dataset.append(row)
    read.close()
    return dataset

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer(char_level=False)
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
    global prediksi
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    prediksi = prediction
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)

    return ' '.join(target)

# translate
def translate(model, tokenizer, sources):
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, all_tokenizer, source)
    return translation

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    result_precision = precision(y_true, y_pred)
    result_recall = recall(y_true, y_pred)
    return 2*((result_precision*result_recall)/(result_precision+result_recall+K.epsilon()))

# load datasets
dataset = load_clean_sample_data()
dataset = np.reshape(dataset, (-1,2))
dataset1 = dataset.reshape(-1,1)

# prepare tokenizer
all_tokenizer = create_tokenizer(dataset1[:,0])
all_vocab_size = len(all_tokenizer.word_index) + 1
all_length = max_length(dataset1[:, 0])

# load model
model = load_model('/content/Model/MODEL_LSTM_LSTM_ATTNDECODER.h5',
                   custom_objects={'AttentionDecoder': AttentionDecoder,
                                   'precision':precision, 'recall':recall, 'f1':f1})

while(True):
    q = (input(str("You: ")))
    user = q
    chat_data = q + "\t"
    if user == 'bye':
      break
    q = q.strip().split('\n')
    #we tokenize
    X = all_tokenizer.texts_to_sequences(q)
    X = pad_sequences(X, maxlen=all_length, padding='post')

    # find reply and print it out
    a = translate(model, all_tokenizer, X)
    words = a.split()
    chat_data2 = (""+ " ".join(sorted(set(words), key=words.index)) + '\n')
    print ('Bot: ' + " ".join(sorted(set(words), key=words.index)))
    with open('chats.tsv', 'a') as file:
        file.write(chat_data + "\t" + chat_data2 )
print("Simpan Chat tu? Taip 'au' tauka 'enda'. File chat enda disimpan enti nuan madah 'enda'.")
choice = input("Pilih: ")
if choice=='enda':
  if os.path.exists("chats.tsv"):
    os.remove("chats.tsv")
    print('File chat enda disimpan.')
else:
  print('File chat disimpan ba "chats.tsv".')

In [ ]:
pip install flask-ngrok

In [ ]:
import numpy as np #app.py
import csv
import os
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from attention_decoder import AttentionDecoder
from keras import backend as K

file = "Iban.tsv"
folder = "Corpus"


# load a clean dataset
def load_clean_sample_data():
    #return load(open(filename, 'rb'))
    filepath = os.path.join(folder,file)
    with open(filepath, "r", encoding="utf8") as read:
        reader = csv.reader(read,delimiter="\t")
        dataset = []
        for row in reader:
            dataset.append(row)
    read.close()
    return dataset

# fit a tokenizer
def create_tokenizer(lines):
  tokenizer = Tokenizer(char_level=False)
  tokenizer.fit_on_texts(lines)
  return tokenizer

# max sentence length
def max_length(lines):
  return max(len(line.split()) for line in lines)

# map an integer to a word
def word_for_id(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
    if index == integer:
      return word
  return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
    global prediksi
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    prediksi = prediction
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)

    return ' '.join(target)

# translate
def translate(model, tokenizer, sources):
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, all_tokenizer, source)
    return translation

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    result_precision = precision(y_true, y_pred)
    result_recall = recall(y_true, y_pred)
    return 2*((result_precision*result_recall)/(result_precision+result_recall+K.epsilon()))

def reply(model, input_text):
    if len(input_text) > MAX_INPUT_SEQ_LENGTH:
        input_text = input_text[0:MAX_INPUT_SEQ_LENGTH]
    input_seq = np.zeros((1, self.max_encoder_seq_length, self.num_encoder_tokens))
    for idx, char in enumerate(input_text.lower()):
        if char in self.input_char2idx:
            idx2 = self.input_char2idx[char]
            input_seq[0, idx, idx2] = 1
    states_value = self.encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, self.num_decoder_tokens))
    target_seq[0, 0, self.target_char2idx['\t']] = 1
    target_text = ''
    terminated = False
    while not terminated:
        output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)

        sample_token_idx = np.argmax(output_tokens[0, -1, :])
        sample_character = self.target_idx2char[sample_token_idx]
        target_text += sample_character

        if sample_character == '\n' or len(target_text) >= self.max_decoder_seq_length:
            terminated = True

        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        target_seq[0, 0, sample_token_idx] = 1
        states_value = [h, c]
    return target_text

# load datasets
dataset = load_clean_sample_data()
dataset = np.reshape(dataset, (-1,2))
dataset1 = dataset.reshape(-1,1)

# prepare tokenizer
all_tokenizer = create_tokenizer(dataset1[:,0])
all_vocab_size = len(all_tokenizer.word_index) + 1
all_length = max_length(dataset1[:, 0])

# load model
model = load_model('/content/Model/MODEL_LSTM_LSTM_ATTNDECODER.h5',
                  custom_objects={'AttentionDecoder': AttentionDecoder,
                                  'precision':precision, 'recall':recall, 'f1':f1})
bot_conversations = []


from flask_ngrok import run_with_ngrok
from flask import Flask, request, send_from_directory, redirect, render_template, flash, url_for, jsonify,make_response, abort

app = Flask(__name__)
app.config.from_object(__name__)  # load config from this file , flaskr.py
run_with_ngrok(app)
# Load default config and override config from an environment variable
app.config.from_envvar('FLASKR_SETTINGS', silent=True)
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024


@app.route('/')
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return 'About Us'

@app.route('/reply', methods=['POST', 'GET'])
def reply():
    if request.method == 'POST':
        if 'sentence' not in request.form:
            flash('No sentence post')
            redirect(request.url)
        elif request.form['sentence'] == '':
            flash('No sentence')
            redirect(request.url)
        elif request.form['sentence'] == 'bye':
            bot_conversations.append('Bot: ' + 'Randau tu automatik disimpan dalam chats.tsv. Nuan ka muai randau tu ari disimpan? Taip "au" ')
        elif request.form['sentence'] == 'au':
          if os.path.exists("chats.tsv"):
            os.remove("chats.tsv")
            bot_conversations.append('Bot: ' + 'Fail Chat udah dibuai.')
            bot_conversations.append('Bot: ' + 'Bye. Ila berandau baru.')
        else:
            sent = request.form['sentence']
            chat_data = sent + "\t"
            bot_conversations.append('You: ' + sent)
            sent = sent.strip().split('\n')
            X = all_tokenizer.texts_to_sequences(sent)
            X = pad_sequences(X, maxlen=all_length, padding='post')


            # find reply and print it out
            a = translate(model, all_tokenizer, X)
            #a = set(a)
            words = a.split()
            #print('ANSWER: %s' % (thing))
            bot_conversations.append('Bot: ' + " ".join(sorted(set(words), key=words.index)))
            chat_data2 = (""+ " ".join(sorted(set(words), key=words.index)) + '\n')
            with open('chats.tsv', 'a') as file:
              file.write(chat_data + "\t" + chat_data2 )


    return render_template('reply.html', conversations=bot_conversations)

def main():
    app.run()

if __name__ == '__main__':
    main()

Inputs Shape: (None, 15, 256)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bae4ed3ddde9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Jun/2021 17:55:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:55:15] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:55:15] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Jun/2021 17:55:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:55:17] "GET /reply HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:55:17] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:55:18] "GET /reply HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:55:20] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:55:24] "POST /reply HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:55:41] "POST /reply HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:55:50] "POST /reply HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:55:57] "POST /reply HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2021 17:56:03] "POST /reply HTTP/1.1" 200 -
